# Pregunta 2

### a) En primer lugar se crea el dataframe y se analizan sus registros.

In [67]:
import urllib
import pandas as pd
train_data_url = "http://www.inf.utfsm.cl/~jnancu/stanford-subset/polarity.train"
test_data_url = "http://www.inf.utfsm.cl/~jnancu/stanford-subset/polarity.dev"
train_data_f = urllib.urlretrieve(train_data_url, "train_data.csv")
test_data_f = urllib.urlretrieve(test_data_url, "test_data.csv")
ftr = open("train_data.csv", "r")
fts = open("test_data.csv", "r")
rows = [line.split(" ",1) for line in ftr.readlines()]
train_df = pd.DataFrame(rows, columns=['Sentiment','Text'])
train_df['Sentiment'] = pd.to_numeric(train_df['Sentiment'])
rows = [line.split(" ",1) for line in fts.readlines()]
test_df = pd.DataFrame(rows, columns=['Sentiment','Text'])
test_df['Sentiment'] = pd.to_numeric(test_df['Sentiment'])
print "Dimensiones training set: ", train_df.shape
print "Dimensiones testing set: ", test_df.shape
print "----------------------------------------------------------"
print  "Conteo de registros de cada clase en el training set"
print train_df['Sentiment'].value_counts()
print "----------------------------------------------------------"
print  "Conteo de registros de cada clase en el testing set"
print test_df['Sentiment'].value_counts()
#print train_df.head()
#print test_df.tail()

Dimensiones training set:  (3554, 2)
Dimensiones testing set:  (3554, 2)
----------------------------------------------------------
Conteo de registros de cada clase en el training set
-1    1784
 1    1770
Name: Sentiment, dtype: int64
----------------------------------------------------------
Conteo de registros de cada clase en el testing set
-1    1803
 1    1751
Name: Sentiment, dtype: int64


### b) En esta sección se pre procesa el texto mediante la utilización de las operaciones lower-casing, y stemming. Para ello se crea la función word_extractor.

In [68]:
import re, time
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer, word_tokenize
from nltk.stem.porter import PorterStemmer
def word_extractor(text,stop = 1):
    #wordlemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    commonwords = stopwords.words('english')
    text = re.sub(r'([a-z])\1+', r'\1\1',text)#substitute multiple letter by two
    words = ""
    '''wordtokens = [ wordlemmatizer.lemmatize(word.lower()) \
                  for word in word_tokenize(text.decode('utf-8', 'ignore')) ]'''
    wordtokens = [ stemmer.stem(word.lower()) \
                  for word in word_tokenize(text.decode('utf-8', 'ignore')) ]
    for word in wordtokens:
        if stop==1:
            if word not in commonwords:
                words+=" "+word
        else:
            words+=" "+word
    return words
print word_extractor("I love to eat cake")
print word_extractor("I love eating cake")
print word_extractor("I loved eating the cake")
print word_extractor("I do not love eating cake")
print word_extractor("I dont love eating cake")
print word_extractor("I like learning in  university")
print word_extractor("The cats are playing with fishes")
print word_extractor("He believes in miracles")

 love eat cake
 love eat cake
 love eat cake
 love eat cake
 dont love eat cake
 like learn univers
 cat play fish
 believ miracl


En el caso anterior, el stemming lleva todas las palabras a su raíz sin importar el contexto en donde estas se encuentren. Stemming no puede discriminar, y solo se basa en ver las conjugaciones de las palabras de forma heurísticas, y dependiendo de la terminación es como decide a que raíz pertenece. Esto puede ocasionar problemas como por ejemplo con la palabra university, en donde el proceso de stemming la transforma a univers y puede ocasionar perdida de informacion ya que cambia el significado.

Si no se realiza stemming, las palabras quedarían conjugadas y para un procesador de texto o clasificador, no serían equivalentes aunque estén en un contexto similar. El problema es entonces que se generarían mas palabras, con lo cual aumenta el vocabulario y por ende la complejidad del dataset.

Con respecto a los "stop words", estos se definen en un conjunto de articulos, pronombres y preposiciones, y son quitados de las cadenas de texto, ya que son palabras muy comunes y que no aportan significado al texto, aunque en algunas aplicaciones si pueden llegar a ser utilizadas.

### c) En este caso se utiliza un lemmatizer para comparar con los resultados anteriores.

In [69]:
def word_extractor2(text, stop=1):
    wordlemmatizer = WordNetLemmatizer()
    commonwords = stopwords.words('english')
    text = re.sub(r'([a-z])\1+', r'\1\1',text)#substitute multiple letter by two
    words = ""
    wordtokens = [ wordlemmatizer.lemmatize(word.lower()) \
                  for word in word_tokenize(text.decode('utf-8','ignore')) ]
    for word in wordtokens:
        if stop==1:
            if word not in commonwords:
                words+=" "+word
        else:
            words+=" "+word
    return words

print word_extractor2("I love to eat cake")
print word_extractor2("I love eating cake")
print word_extractor2("I loved eating the cake")
print word_extractor2("I do not love eating cake")
print word_extractor2("I dont love eating cake")
print word_extractor2("I like learning in  university")
print word_extractor2("The cats are playing with fishes")
print word_extractor2("He believes in miracles")

 love eat cake
 love eating cake
 loved eating cake
 love eating cake
 dont love eating cake
 like learning university
 cat playing fish
 belief miracle


Procesando las cadenas mediante el lemmatizer, se conserva muchas veces la conjugación de las palabras según el contexto, y también son llevadas a su tronco léxico. Por ejemplo las palabras loved y eating se conservan, y en este caso university se mantiene igualmente, no así en el proceso de stemming. Entonces la lematización es un proceso mucho más complejo, que no solo revisa cada palabra y su terminación, sino que además observa el contexto para realizar la agrupación de palabras similares.

### d) 

In [70]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
# Se crean las listas con los textos procesados
texts_train = [word_extractor2(text) for text in train_df.Text]
texts_test = [word_extractor2(text) for text in test_df.Text]
vectorizer = CountVectorizer(ngram_range=(1, 1), binary='False')
vectorizer.fit(np.asarray(texts_train))
#Se crea la matriz con la cuenta de los tokens(test y train)
features_train = vectorizer.transform(texts_train)
features_test = vectorizer.transform(texts_test)
print "Dimension conjunto de entrenamiento: ",features_train.shape
print "Dimension conjunto de pruebas: ", features_test.shape
print "--------------------------------------------------------------"
#label 0 y 1
labels_train = np.asarray((train_df.Sentiment.astype(float)+1)/2.0)
labels_test = np.asarray((test_df.Sentiment.astype(float)+1)/2.0)
vocab = vectorizer.get_feature_names()
dist_tr=list(np.array(features_train.sum(axis=0)).reshape(-1,))
dist_ts=list(np.array(features_test.sum(axis=0)).reshape(-1,))
#for tag, count in zip(vocab, dist):
#    print count, tag
index_tr= sorted(range(len(dist_tr)), key=lambda k: dist_tr[k], reverse=True )
index_ts= sorted(range(len(dist_ts)), key=lambda k: dist_ts[k], reverse=True )
#print sorted(dist_tr, reverse=True)[0]
print "10 Palabras mas frecuentes \n"
print "{0:<30}{1} \n".format("Conjunto de entrenamiento","Conjunto de pruebas")
for i in range (0,10):
    m1=index_tr[i]
    m2=index_ts[i]
    print '{0:<5}{1:<25}{2:<5}{3}'.format(dist_tr[m1], vocab[m1], dist_ts[m2], vocab[m2])

Dimension conjunto de entrenamiento:  (3554, 9663)
Dimension conjunto de pruebas:  (3554, 9663)
--------------------------------------------------------------
10 Palabras mas frecuentes 

Conjunto de entrenamiento     Conjunto de pruebas 

566  film                     558  film
481  movie                    540  movie
246  one                      250  one
245  like                     238  ha
224  ha                       230  like
183  make                     197  story
176  story                    175  character
163  character                165  time
145  comedy                   161  make
143  even                     134  comedy


Se aprecia en la tabla anterior, que el vocabulario se compone de 9663 palabras o tokens,  y que tanto los set de entrenamiento y prueba contienen 3554 registros o "reviews" de usuarios. Las 10 palabras más frecuentes se repiten en gran medida en ambos conjuntos y conservan un orden similar, esto refleja que los datos presentan estructuras similares.

### e)

In [71]:
from sklearn.metrics import classification_report
def score_the_model(model,x,y,xt,yt,text):
    acc_tr = model.score(x,y)
    acc_test = model.score(xt[:-1],yt[:-1])
    print "Training Accuracy %s: %f"%(text,acc_tr)
    print "Test Accuracy %s: %f"%(text,acc_test)
    print "Detailed Analysis Testing Results ..."
    print(classification_report(yt, model.predict(xt), target_names=['+','-']))

La función $\textit{classification_report}$  imprime 3 métricas las cuales son precision, recall y F1 score.
$\textit{Precision}$ se define como:

$$ precision = \frac{tp}{tp+fp}$$ 

Donde tp representa los verdaderos positivos y fp los falsos positivos. Intuitivamente es la habilidad del clasificador para no etiquetar como positivo una muestra que es negativa. Su mejor valor es 1 y su peor valor es 0.

Recall score se define como: 

$$ recall = \frac{tp}{tp+fp}$$ 

Donde tp representa los verdaderos positivos y fp los falsos negativos. Recall representa la habilidad del clasificador para encontrar todos los ejemplos positivos. Su mejor valor es 1 y su peor valor es 0.

Entonces $\textit{precision}$ representa la exactitud o calidad del clasificador y $\textit{recall}$ la medida de completitud o cantidad.

Finalmente, F1 es una medida que utiliza las dos métricas anteriores en un promedio ponderado. Su fórmula es: 

$$ f1 = 2* \frac{precision * recall}{precision+recall}$$


### f) Para el caso de Bernoulli naive bayes los features deben tener valor 0 o 1, esto lo hace automáticamente BernoulliNB de la librería sklearn.

El siguiente código crea los textos y features sin stop words y además otro utilizando stemming, para posteriormente ser analizados.

In [72]:
# El caso comun en el puntop (d) es lemmatizer filtrando stopwords
# Se crea training y testing set sin filtrar stopwords mediante lemmatizer.

texts_train_noSW = [word_extractor2(text, 0) for text in train_df.Text]
texts_test_noSW = [word_extractor2(text,0) for text in test_df.Text]
vectorizer = CountVectorizer(ngram_range=(1, 1), binary='False')
vectorizer.fit(np.asarray(texts_train_noSW))
#Se crea la matriz con la cuenta de los tokens(test y train)
features_train_noSW = vectorizer.transform(texts_train_noSW)
features_test_noSW = vectorizer.transform(texts_test_noSW)

# Se crea training y testing set utilizando stemming

texts_train_stem = [word_extractor(text) for text in train_df.Text]
texts_test_stem = [word_extractor(text) for text in test_df.Text]
vectorizer = CountVectorizer(ngram_range=(1, 1), binary='False')
vectorizer.fit(np.asarray(texts_train_stem))
#Se crea la matriz con la cuenta de los tokens(test y train)
features_train_stem = vectorizer.transform(texts_train_stem)
#print features_train_stem.shape
features_test_stem = vectorizer.transform(texts_test_stem)


##### Modelo con lemmatizer y filtrando stop words Bernoulli NB

In [73]:
from sklearn.naive_bayes import BernoulliNB
import random
def do_NAIVE_BAYES(x,y,xt,yt):
    model = BernoulliNB()
    model = model.fit(x, y)
    score_the_model(model,x,y,xt,yt,"BernoulliNB")
    return model
model=do_NAIVE_BAYES(features_train,labels_train,features_test,labels_test)

Training Accuracy BernoulliNB: 0.958638
Test Accuracy BernoulliNB: 0.738531
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.75      0.73      0.74      1803
          -       0.73      0.75      0.74      1751

avg / total       0.74      0.74      0.74      3554



La tabla anterior muestra que el desempeño de Bernoulli Naive Bayes no es del todo bueno, ya que presenta una alta exactitud en el set de entrenamiento, pero mucho peor en el de pruebas, dando entender que el modelo esta sobre ajustando los datos. Esto también queda en evidencia con los otros indicadores como precision y recall, de donde la calidad para clasificar no es muy alta, y tampoco es completa. Esto puede deberse principalmente a que no se toma en cuenta la cantidad de palabras o tokens en cada registro, sino que son transformados a 0 o 1, perdiendo así información.

##### Modelo con lemmatizer sin filtrar stop words Bernoulli NB

In [74]:
model_noSW=do_NAIVE_BAYES(features_train_noSW,labels_train,features_test_noSW,labels_test)

Training Accuracy BernoulliNB: 0.955262
Test Accuracy BernoulliNB: 0.748663
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.76      0.74      0.75      1803
          -       0.74      0.76      0.75      1751

avg / total       0.75      0.75      0.75      3554



Aquí no se presenta gran mejora, el modelo sigue sobre ajustando, pero al no considerarse el número de stop words, sino que solo se interpreta como si aparece o no la palabra, no afecta demasiado a los resultados. Una ventaja es que requiere menos procesamiento de cómputo y se obtiene levemente un mejor resultado.

##### Modelo con stemmer Bernoulli NB

In [75]:
model_stem = do_NAIVE_BAYES(features_train_stem,labels_train,features_test_stem,labels_test)

Training Accuracy BernoulliNB: 0.942881
Test Accuracy BernoulliNB: 0.747819
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.76      0.74      0.75      1803
          -       0.74      0.75      0.75      1751

avg / total       0.75      0.75      0.75      3554



Al igual que el caso anterior, la mejora es muy pequeña, sin embargo al usar stemming el procesamiento de los datos es más rápido ya que no debe interpretar el contexto y además genera muchos menos features que lemmatization, 1700 palabras menos en el vocabulario aproximadamente. Esto muestra que bernoulli no se ve afectado por la cantidad de features en gran medida.

##### Analisis exploratorio Bernoulli NB

In [76]:
test_pred = model.predict_proba(features_test)
#print model.classes_
spl = random.sample(xrange(len(test_pred)), 5)
for text, sentiment in zip(test_df.Text[spl], test_pred[spl]):
    print sentiment, text

[ 0.26622002  0.73377998] you wouldn't want to live waydowntown , but it is a hilarious place to visit .

[ 0.17491976  0.82508024] what makes it worth watching is quaid's performance .

[ 0.55956959  0.44043041] both the crime story and the love story are unusual . but they don't fit well together and neither is well told .

[ 0.60586349  0.39413651] this overproduced and generally disappointing effort isn't likely to rouse the rush hour crowd .

[ 0.9894078  0.0105922] it's another video movie photographed like a film , with the bad lighting that's often written off as indie film naturalism .

